In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [3]:
pml = pd.read_csv('pml_train.csv')
print('data loaded')
pml.head()

data loaded


,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,0,A,A,A,A,B,A,A,A,A,...,0.669748,0.515940,0.55421,0.53774,0.47225,0.492200,0.481306,0.756454,0.344502,2152.80
1,1,B,A,A,A,A,B,A,A,A,...,0.484775,0.698278,0.35533,0.40657,0.40666,0.468839,0.458493,0.304350,0.470455,1019.89
2,2,A,A,A,B,A,A,A,A,A,...,0.350956,0.363768,0.58354,0.44352,0.39599,0.341813,0.352251,0.339244,0.283969,4477.83
3,3,B,A,A,A,A,B,A,A,A,...,0.748243,0.538694,0.54829,0.50420,0.51111,0.711942,0.698722,0.709578,0.776114,907.11
4,4,A,B,A,A,A,A,A,A,B,...,0.321058,0.326430,0.31280,0.39648,0.38016,0.245410,0.241676,0.261150,0.342082,974.62


In [4]:
pml_train_y = pml.loss
pml_train_X = pml.drop(columns=['id', 'loss'])
pml_train_X.shape

(131822, 130)

In [5]:
pml_train_d_X = pd.get_dummies(pml_train_X)
pml_train_d_X.shape

(131822, 1111)

In [6]:
pml_test = pd.read_csv('pml_test_features.csv')
pml_test_X = pml_test.drop(columns=['id'])
pml_test_X.shape

(56496, 130)

In [7]:
# # one-hot encoded test
# pml_test_d_X = pd.get_dummies(pml_test)
# pml_test_d_X.shape

In [8]:
# cDrop = [c for c in pml_test_d_X.columns if c not in pml_train_d_X.columns]
# print(cDrop)
# pml_test_d_X.drop(columns = cDrop, inplace=True)

# for c in pml_train_d_X.columns:
#     if c not in pml_test_d_X.columns:
#         pml_test_d_X[c] = 0
# print(pml_test_d_X.shape)
# pml_test_d_X.head()

In [9]:
# pml_train_cont = pml_train_X.filter(regex=("cont\d*"))
# pml_train_cont.head()

In [10]:
#Generate a correlation matrix between features
# x_corr = pml_train_cont.corr()
# mask = np.zeros_like(x_corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True
# f, ax = plt.subplots(figsize=(11, 9))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# sns.heatmap(x_corr, mask=mask, vmax=1, cmap=cmap, center=0,
#             square=True, linewidths=.5)

In [11]:
def col_op(col):
    return col.astype('category').cat.codes

In [12]:
def digit_op(code):
    num = 0
    for alpha in code:
        num *= 26
        num += ord(alpha) - ord('A') + 1
    return num

def to_digit(col):
    return col.apply(digit_op)

In [13]:
to_digit(pd.Series(['AA','BB','A']))

0    27
1    54
2     1
dtype: int64

In [14]:
def encode_X_to_digit(orig_train_X):
    c_X = orig_train_X.copy()
    tmp = orig_train_X.select_dtypes(exclude=['float64','int64'])
    # tmp = pd.Categorical(tmp)

    c_X.loc[:, tmp.columns] = tmp.apply(lambda col: col.astype('category').cat.codes)
    return c_X

In [29]:
pml_train_c_X = encode_X_to_digit(pml_train_X)
pml_test_c_X = encode_X_to_digit(pml_test_X)
print('data encoded')

data encoded


In [16]:
pml_train_c_X.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0,0,0,0,1,0,0,0,0,0,...,0.577339,0.669748,0.515940,0.55421,0.53774,0.47225,0.492200,0.481306,0.756454,0.344502
1,1,0,0,0,0,1,0,0,0,0,...,0.281143,0.484775,0.698278,0.35533,0.40657,0.40666,0.468839,0.458493,0.304350,0.470455
2,0,0,0,1,0,0,0,0,0,0,...,0.499798,0.350956,0.363768,0.58354,0.44352,0.39599,0.341813,0.352251,0.339244,0.283969
3,1,0,0,0,0,1,0,0,0,0,...,0.281143,0.748243,0.538694,0.54829,0.50420,0.51111,0.711942,0.698722,0.709578,0.776114
4,0,1,0,0,0,0,0,0,1,0,...,0.491114,0.321058,0.326430,0.31280,0.39648,0.38016,0.245410,0.241676,0.261150,0.342082


In [17]:
# #Generate a correlation matrix between features
# x_corr = pml_train_c_X.corr()
# mask = np.zeros_like(x_corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True
# f, ax = plt.subplots(figsize=(22, 18))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# sns.heatmap(x_corr, mask=mask, vmax=1, cmap=cmap, center=0,
#             square=True, linewidths=.5)

In [31]:
# normalize
normalizer = MinMaxScaler().fit(pml_train_c_X)
pml_train_c_X = normalizer.transform(pml_train_c_X)
pml_test_c_X = normalizer.transform(pml_test_c_X)

pml_test_c_X.shape

(56496, 130)

In [32]:
# test/train split
val_train_X, val_test_X, val_train_y, val_test_y = train_test_split(pml_train_c_X, pml_train_y, random_state=2018, test_size=0.05)
print('data prep finished')

data prep finished


In [20]:
# pipeline:
def try_clf(clf, clf_name=''):
    print('start training ' + clf_name )
    clf.fit(val_train_X, val_train_y)
    print(clf_name + ' train :' + str(clf.score(val_train_X, val_train_y)))
    print(clf_name + ' test  :' + str(clf.score(val_test_X, val_test_y)))
    
    return clf

In [21]:
def use_clf(clf, clf_name='clf'):
    ans = clf.predict(pml_test_c_X)
    filename = clf_name + '.csv'
    pd.DataFrame({'id':pml_test['id'], 
          'loss':ans}).to_csv(filename,index = False)
    print('exported as ' + filename)

## Linear

In [22]:
lin = LinearRegression()
lin = try_clf(lin)

start training 


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


 train :0.4087461350680288
 test  :0.38260017823378595


In [23]:
use_clf(lin, 'lin_default')

exported as lin_default.csv


## SVR

In [36]:
svr = SVR(C=1.0, max_iter = 1000)
svr = try_clf(svr, 'svr rbf')

start training svr rbf


/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


svr rbf train :-2.276566690358891
svr rbf test  :-2.3657263879438717


In [25]:
use_clf(svr,'svr_default')

exported as svr_default.csv


## tree

In [20]:
#. ec2-spotter/fast_ai/create_vpc.sh

In [ ]:
rf = RandomForestRegressor(n_estimators=80, max_leaf_nodes = 8, 
                            random_state = 2018)
rf = try_clf(rf, 'rf')

start training rf


In [39]:
use_clf(rf,'rf_10tree_5maxnodes')

exported as rf_10tree_5maxnodes.csv


In [ ]:
c = 'AB'

In [ ]:
for a in reversed(c):
    print(a)

In [30]:
temp = pd.DataFrame({'a':['A','B','C','B'],
             'b':['DC','BD','BF','CC']})
temp

,a,b
0,A,DC
1,B,BD
2,C,BF
3,B,CC


In [31]:
def op(col):
    return col.astype('category').cat.codes

In [39]:
# uniques = np.sort(pd.unique(temp.values.ravel()))
# temp.apply(lambda x: x.astype('category', categories=uniques))

le = LabelEncoder()
le.fit(temp.values.flat)

# Convert to digits.
temp = temp.apply(le.transform)
temp

NameError: name 'LabelEncoder' is not defined